In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function linefragments(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            for k in sigma[h]
                line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[k], β)
                    end
                end
            end
        end
    end
    fragparams = []
    for line in params
        push!(line, 0.0, 1.0)
        line = sort(collect(Set(line)))
        push!(fragparams, line)
    end
    return fragparams
end

linefragments (generic function with 1 method)

## Dipendenze della funzione

In [3]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        return ()
    end
    return α,β
end

intersection (generic function with 1 method)

## Dati in input

In [4]:
b=[[],[]]
EV=[[1,1]]

for i=1:60
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(EV,[1+4*(i-1),2+4*(i-1)])
           push!(EV,[1+4*(i-1),3+4*(i-1)])
           push!(EV,[2+4*(i-1),4+4*(i-1)])
           push!(EV,[3+4*(i-1),4+4*(i-1)])
end

V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
filter!(e->e!=[1,1],EV)


using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

function coordintervals(coord,bboxes)
    boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
    for (h,box) in enumerate(bboxes)
        key = box[coord,:]
        if haskey(boxdict,key) == false
            boxdict[key] = [h]
        else
            push!(boxdict[key], h)
        end
    end
    return boxdict
end

function boxcovering(bboxes, index, tree)
    covers = [[] for k=1:length(bboxes)]
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        for x in iterator
            append!(covers[i],x.value)
        end
    end
    return covers
end

function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
           V,CV = model[1:2]
           # se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
           dim = size(V,1)
           cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
           #----------------------------------------------------------
           bboxes = [hcat(boundingbox(cell)...) for cell in cellpoints]
           xboxdict = coordintervals(1,bboxes)
           yboxdict = coordintervals(2,bboxes)
           # xs,ys are IntervalTree type
           xs = IntervalTrees.IntervalMap{Float64, Array}()
           for (key,boxset) in xboxdict
               xs[tuple(key...)] = boxset
           end
           ys = IntervalTrees.IntervalMap{Float64, Array}()
           for (key,boxset) in yboxdict
               ys[tuple(key...)] = boxset
           end
           xcovers = boxcovering(bboxes, 1, xs)
           ycovers = boxcovering(bboxes, 2, ys)
           covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

           if dim == 3
               zboxdict = coordintervals(3,bboxes)
               zs = IntervalTrees.IntervalMap{Float64, Array}()
               for (key,boxset) in zboxdict
                   zs[tuple(key...)] = boxset
               end
               zcovers = boxcovering(bboxes, 3, zs)
               covers = [intersect(pair...) for pair in zip(zcovers,covers)]
           end
           # remove each cell from its cover
           for k=1:length(covers)
               covers[k] = setdiff(covers[k],[k])
           end
           return covers
       end

Sigma = spaceindex((V,EV))


240-element Array{Array{Int64,1},1}:
 [2, 3]
 [1, 4]
 [1, 4, 5]
 [2, 6, 3]
 [6, 3, 7]
 [4, 5, 8]
 [5, 8, 9]
 [6, 10, 7]
 [10, 7, 11]
 [8, 9, 12]
 [9, 12, 13]
 [10, 14, 11]
 [14, 11, 15]
 ⋮
 [230, 227, 231]
 [228, 229, 232]
 [229, 232, 233]
 [230, 234, 231]
 [234, 231, 235]
 [232, 233, 236]
 [233, 236, 237]
 [234, 238, 235]
 [238, 235, 239]
 [236, 237, 240]
 [237, 240]
 [238, 239]

## 0) Benchmark vecchia funzione 

In [5]:
@btime linefragments(V,EV,Sigma)

  1.936 ms (34634 allocations: 1.25 MiB)


240-element Array{Any,1}:
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 ⋮
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]

## 1) Controllo se la funzione è type unstable

In [6]:
@code_warntype linefragments(V,EV,Sigma)

Variables
  #self#::Core.Compiler.Const(linefragments, false)
  V::Array{Float64,2}
  EV::Array{Array{Int64,1},1}
  Sigma::Array{Array{Int64,1},1}
  m::Int64
  sigma::Array{Array{Int64,1},1}
  reducedsigma::Array{Array{Int64,1},1}
  params::Array{Array{Float64,1},1}
  @_9::Union{Nothing, Tuple{Int64,Int64}}
  fragparams::Array{Any,1}
  @_11::Union{Nothing, Tuple{Array{Float64,1},Int64}}
  @_12::Array{Array{Float64,1},1}
  @_13::Int64
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  h::Int64
  line1::Array{Float64,2}
  @_18::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_20::Int64
  line2::Array{Float64,2}
  out::Tuple
  α::Any
  β::Any
  line::Array{Float64,1}
  @_26::Any
  @_27::Any
  @_28::Any

Body::Array{Any,1}
1 ──        Core.NewvarNode(:(params))
│           Core.NewvarNode(:(@_9))
│           Core.NewvarNode(:(fragparams))
│           Core.NewvarNode(:(@_11))
│           (m = Main.length(Sigma))
│           (sigma = Main.map(Main.sort, Sigma))
│           (reducedsigm

la funzione non  type unstable in quanto ho nell'output la stringa:

    Body::Array{Any,1}

## 2) Revisione del codice e utilizzo dei threads  
  
Abbiamo migliorate le performance del codice applicando i seguenti passi:  
  
1) Abbiamo convertito la list comprehension della creazione dei vettori 'params' in un ciclo for in modo da poter 
utilizzare la macro @threads (in quanto task parallelizzabile)  
2) Abbiamo allocato line1 e line2 fuori dal for in modo tale che non vengano distrutti e riallocati ad ogni iterazione  
3) Abbiamo eliminato la variabile 'fragparams' presente nell'ultima porzione di codice, in quanto riallocava la stessa informazione contenuta in 'params' eliminando solo i doppioni. Inoltre abbiamo aggiunto la macro threads a tutte le task parallelizzabili.  
4) Abbiamo convertito l'ultimo for each in un for i=1:n.....

In [7]:
using Base.Threads
function linefragments2(V,EV,sigma)
    m = length(sigma) 
    sigma = map(sort,sigma) 
    params = Array{Array{Float64,1}}(undef,m)
    @threads for i=1:m
        params[i] = []
    end
    line1=[0.0 0.0; 0.0 0.0]
    line2=[0.0 0.0; 0.0 0.0]
    @threads for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            @threads for k in sigma[h]
            line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    if 0<=out[1]<=1 && 0<=out[2]<=1
                        push!(params[h], out[1])
                        push!(params[k], out[2])
                    end
                end
            end
        end
        end
    len = length(params)
    @threads for i=1:len
        push!(params[i], 0.0, 1.0)
        params[i] = sort(collect(Set(params[i])))
    end
    return params
end
print("Numero di threads allocati :")
println(nthreads())
@btime linefragments2(V,EV,Sigma)

Numero di threads allocati :2
  1.102 ms (31024 allocations: 1.11 MiB)


240-element Array{Array{Float64,1},1}:
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 ⋮
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.6666666666666666, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 0.3333333333333333, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]